# Training & Evaluation 
* In this notebook we are going to train a simple object detection CNN from scratch. 
* We'll reuse similar CNN that we used for image classification. 
* We'll use the optimized data generation script to generate the training data. 

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import mnist
import matplotlib.pyplot as plt
import matplotlib.patches as patches



## validate tensorflow 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Constants

In [4]:
data_dir = Path("..","data")
models_dir = Path("..","models")

In [5]:
import os
import sys
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import graph_compatible_data_generator,training_utils

## logic to auto reload scripts without restarting the kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize Pipeline

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_tensor = tf.convert_to_tensor(x_train, dtype=tf.float32)
# X_tensor = tf.reshape(X_tensor, shape=(-1, 28, 28, 1))
y_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)

batch_size = 32
raw_dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))

# create a generator for 2 digits
data_gen_2_digits = graph_compatible_data_generator.create_data_generator(2)

processed_dataset_2 = raw_dataset.map(
    data_gen_2_digits).batch(batch_size=batch_size).prefetch(tf.data.AUTOTUNE)

### Validating Pipeline

In [11]:
def visualize_generated_data(batch):
    batched_canvases,predictions = batch
    print(f"batched_canvases shape: {batched_canvases.shape}")
    print(f"predictions shape ", tf.shape(predictions))
    # Get the very first canvas from the batch (shape 100x100)
    # We use .numpy() to convert it from a EagerTensor to a NumPy array for plotting
    canvas_to_show = batched_canvases[0].numpy()
    prediction = predictions[0]

    print(f"Canvas shape: {canvas_to_show.shape}")
    print(f"Single Prediction shape: {prediction.shape}")
    
    
    # # Plot it
    # # --- Create a figure and axis ---
    fig, ax = plt.subplots(1, figsize=(8, 8))
    
    # get the 2 predictions
    for i in range(2):
        bbox = (prediction[i]).numpy() * 100
        
        # flag, x_center, y_center, width, height,
        flag = bbox[0]
        x_center = bbox[1]
        y_center = bbox[2]
        width = bbox[3]
        height = bbox[4]
        
        x_min = x_center - (width / 2)
        y_min = y_center - (width / 2)        
        # print("flag, x_center, y_center, width, height",flag, x_min, y_min, width, height,)
        rect = patches.Rectangle(
            (x_min, y_min),
            width,
            height,
            linewidth=2,
            edgecolor='r',
            facecolor='none'
        )
        
        ax.add_patch(rect)
    # Display the image
    ax.imshow(canvas_to_show, cmap='gray')
    
    
    plt.title("Generated 100x100 Canvas (Test 1)")
    plt.show()


# Get one batch
# Your dataset is batched, so .take(1) gets one full batch
# for batch in processed_dataset_2.take(1):
#     visualize_generated_data(batch=batch)

## Initialize The Model

In [17]:
inputs = tf.keras.Input(shape=(100,100,1),batch_size=batch_size ,name="input_layer")

x = tf.keras.layers.Rescaling(scale=1./255, name="rescaling")(inputs)

x = tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=8, kernel_size=5, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)

outputs = tf.keras.layers.Conv2D(filters=45, kernel_size=1, padding='same', activation='linear')(x)

# Define the final model by specifying its inputs and outputs
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (32, 100, 100, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (32, 100, 100, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (32, 100, 100, 8)      │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (32, 100, 100, 8)      │         1,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (32, 50, 50, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (32, 50, 50, 8)        │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (32, 50, 50, 8)        │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (32, 25, 25, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (32, 25, 25, 16)       │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (32, 25, 25, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (32, 12, 12, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (32, 12, 12, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (32, 12, 12, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (32, 6, 6, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (32, 6, 6, 45)         │         1,485 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,845 (85.33 KB)

 Trainable params: 21,845 (85.33 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# step 4: Define the callbacks
checkpoint_filepath = '../models/experiment_1_{epoch:02d}.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
    )

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,clipnorm=1.0),
              loss=training_utils.calculate_model_loss,
              metrics=[training_utils.objectness_metrics, training_utils.bounding_box_metrics, training_utils.classification_metrics])
## step 5: Fit the model
epochs=5

history = model.fit(
  processed_dataset_2,
  epochs=epochs,
  callbacks=[model_checkpoint_callback]
)

Epoch 1/5
----- True Values -----
y_true.shape (None, 5, 15)
----- Pred Values -----
y_pred.shape (None, 6, 6, 45)
anchor_boxes.shape (None, 6, 6, 3, 15)
grid indices shape (None, 5, 2)
selected_anchor_boxes.shape :(None, 5, 3, 15)
anchor_boxes.shape (None, 6, 6, 3, 15)
selected_anchor_boxes.shape :(None, 5, 3, 15)
y_true_boxes.shape (None, 5, 1, 4)
y_pred_boxes.shape (None, 5, 3, 4)
intersection_box_corners.shape (None, 5, 3, 4)
intersection_area.shape (None, 5, 3)
union_area.shape (None, 5, 3)
iou.shape (None, 5, 3)
highest_iou_index.shape (None, 5)
expanded highest_iou_index.shape (None, 5, 1)
best_anchor_boxes.shape (None, 5, 1, 15)
true_values_with_objects.shape : (None, 15)
predicted_values_with_objects.shape : (None, 15)
y_pred_objectness.shape : (None,)
y_pred_bounding_box.shape : (None, 4)
y_pred_classification.shape : (None, 10)
y_true_objectness.shape : (None,)
y_true_bounding_box.shape : (None, 4)
y_true_classification.shape : (None, 10)
Post Activation y_pred_objectness.sh

NotImplementedError: Cannot convert a symbolic tf.Tensor (compile_loss/calculate_model_loss/strided_slice_47:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.